In [1]:
import logging
import os
import random
from collections.abc import Callable
from copy import deepcopy
from dataclasses import dataclass
from datetime import datetime
from json import dumps
from time import time
from typing import Final, Literal

import numpy as np
import structlog
from bayes_opt import BayesianOptimization
from bayes_opt.event import Events
from bayes_opt.logger import JSONLogger
from bayes_opt.util import load_logs
from numpy import ndarray, where
from pandas import DataFrame, Series, concat, read_csv, set_option
from scipy.stats import loguniform, wilcoxon
from sklearn.metrics import (
    f1_score,
)
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.svm import OneClassSVM

set_option("display.max_columns", None)
structlog.configure(
    processors=[
        structlog.stdlib.filter_by_level,
        structlog.stdlib.add_logger_name,
        structlog.stdlib.add_log_level,
        structlog.stdlib.PositionalArgumentsFormatter(),
        structlog.processors.TimeStamper(fmt="iso"),
        structlog.processors.StackInfoRenderer(),
        structlog.processors.format_exc_info,
        structlog.processors.UnicodeDecoder(),
        structlog.processors.JSONRenderer(),
    ],
    context_class=dict,
    logger_factory=structlog.stdlib.LoggerFactory(),
    wrapper_class=structlog.stdlib.BoundLogger,
    cache_logger_on_first_use=True,
)

type ParamGrid = dict[str, tuple[float | str, ...]]
NUM_TRIALS: Final[int] = 10
LOGS_PATH_BAYESIAN: Final[str] = "../reports/logs_bayesian.log"

In [2]:
# Load data
X_train = read_csv("../data/PAMAP2/x_train_data.csv")
X_test = read_csv("../data/PAMAP2/x_test_data.csv")
y_train = read_csv("../data/PAMAP2/y_train_data.csv")
y_test = read_csv("../data/PAMAP2/y_test_data.csv")

X_train["activity"] = y_train
X_test["activity"] = y_test

# Global variables
testing_data: DataFrame
test_targets: Series
training_data: DataFrame
train_targets: Series
MIN_SAMPLES = X_train["activity"].value_counts().sort_values().iloc[0]
MAXIMAZED = False


@dataclass
class SearchResult:
    method: str
    best_params: dict
    best_score: float
    cv_scores: list[float]
    fit_time: float
    n_evaluations: int

In [3]:
def configure_file_logger(filepath: str) -> structlog.BoundLogger:
    """Configure a file-specific logger using structlog."""
    file_handler = logging.FileHandler(filepath, mode="a")
    file_handler.setLevel(logging.INFO)

    structlog.configure(
        processors=[
            structlog.stdlib.filter_by_level,
            structlog.stdlib.add_logger_name,
            structlog.stdlib.add_log_level,
            structlog.stdlib.PositionalArgumentsFormatter(),
            structlog.processors.TimeStamper(fmt="iso"),
            structlog.processors.StackInfoRenderer(),
            structlog.processors.format_exc_info,
            structlog.processors.UnicodeDecoder(),
            structlog.processors.JSONRenderer(),
        ],
        context_class=dict,
        logger_factory=structlog.stdlib.LoggerFactory(),
        wrapper_class=structlog.stdlib.BoundLogger,
        cache_logger_on_first_use=False,
    )
    stdlib_logger = logging.getLogger("hyperparameter_search")
    stdlib_logger.handlers.clear()
    stdlib_logger.addHandler(file_handler)
    stdlib_logger.setLevel(logging.INFO)

    return structlog.get_logger("hyperparameter_search")


def score_function(model: OneClassSVM, Train: DataFrame, test: Series) -> float:
    """
    Objective function to maximize, calcs the F1 score on the test set.
    follows the format needed by scikit-learn's API.
    """
    f1 = f1_score(test_targets, where(model.predict(testing_data) == -1, True, False))
    logger.info(
        {
            "target": f1,
            "params": model.get_params(),
            "datetime": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        }
    )
    return float(f1)


def objective_function_bayesian(nu: float, gamma: float, tol: float) -> float:
    """
    Objective function to optimize F1-Score using Bayesian Optimization.
    """
    global training_data, testing_data, train_targets, test_targets
    oc_svm = OneClassSVM(kernel="rbf", nu=nu, gamma=gamma, tol=tol).fit(training_data)
    return float(
        f1_score(test_targets, where(oc_svm.predict(testing_data) == 1, False, True))
    )

In [4]:
class SimulatedAnnealingSearch:
    """
    Custom Simulated Annealing implementation for hyperparameter optimization.
    """

    def __init__(
        self,
        param_space: dict,
        n_iter: int = NUM_TRIALS,
        initial_temp: float = 1.0,
        cooling_rate: float = 0.95,
        min_temp: float = 0.01,
        random_state: int = 42,
    ):
        self.param_space = param_space
        self.n_iter = n_iter
        self.initial_temp = initial_temp
        self.cooling_rate = cooling_rate
        self.min_temp = min_temp
        self.random_state = random_state
        self.best_params_ = None
        self.best_score_ = -np.inf
        self.cv_results_ = {"mean_test_score": []}

    def _sample_params(self) -> dict:
        """Sample random parameters from the parameter space."""
        return {
            key: values.rvs(random_state=self.random_state)
            if hasattr(values, "rvs")
            else random.choice(values)
            for key, values in self.param_space.items()
        }

    def _neighbor_params(self, current_params: dict) -> dict:
        """Generate neighboring parameters by slightly modifying current ones."""
        neighbor = deepcopy(current_params)
        param_to_modify = random.choice(list(self.param_space.keys()))

        if hasattr(self.param_space[param_to_modify], "rvs"):
            if param_to_modify == "nu":
                current_val = neighbor[param_to_modify]
                neighbor[param_to_modify] = np.clip(
                    current_val + np.random.normal(0, 0.05 * current_val), 0.001, 1.0
                )
            elif param_to_modify == "gamma":
                neighbor[param_to_modify] = 10 ** np.clip(
                    np.log10(neighbor[param_to_modify]) + np.random.normal(0, 0.1),
                    -4,
                    1,
                )
            elif param_to_modify == "tol":
                neighbor[param_to_modify] = 10 ** np.clip(
                    np.log10(neighbor[param_to_modify]) + np.random.normal(0, 0.1),
                    -6,
                    -1,
                )
        else:
            neighbor[param_to_modify] = random.choice(self.param_space[param_to_modify])

        return neighbor

    def _evaluate_params(self, params: ParamGrid, X: DataFrame, y: Series) -> float:
        """Evaluate parameter configuration using cross-validation."""
        return np.mean(
            cross_val_score(
                OneClassSVM(**params, kernel="rbf"), X, y, cv=4, scoring=score_function
            )
        )

    def fit(self, X: DataFrame, y: Series):
        """Fit the simulated annealing search."""
        random.seed(self.random_state)
        np.random.seed(self.random_state)

        current_params = self._sample_params()
        current_score = self._evaluate_params(current_params, X, y)

        self.best_params_ = deepcopy(current_params)
        self.best_score_ = current_score
        temperature = self.initial_temp

        for iteration in range(self.n_iter):
            neighbor_params = self._neighbor_params(current_params)
            neighbor_score = self._evaluate_params(neighbor_params, X, y)
            self.cv_results_["mean_test_score"].append(neighbor_score)

            if neighbor_score > current_score:
                current_params = neighbor_params
                current_score = neighbor_score
            else:
                if (
                    random.random()
                    < np.exp(neighbor_score - current_score / temperature)
                    if temperature > 0
                    else 0
                ):
                    current_params = neighbor_params
                    current_score = neighbor_score

            if current_score > self.best_score_:
                self.best_params_ = deepcopy(current_params)
                self.best_score_ = current_score

            temperature = max(temperature * self.cooling_rate, self.min_temp)

        return self


class GeneticAlgorithmSearch:
    """
    Custom Genetic Algorithm implementation for hyperparameter optimization.
    """

    def __init__(
        self,
        param_space: dict,
        population_size: int = 20,
        n_generations: int = 10,
        mutation_rate: float = 0.1,
        crossover_rate: float = 0.8,
        elite_size: int = 2,
        random_state: int = 42,
    ):
        self.param_space = param_space
        self.population_size = population_size
        self.n_generations = n_generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.elite_size = elite_size
        self.random_state = random_state
        self.best_params_ = None
        self.best_score_ = -np.inf
        self.cv_results_ = {"mean_test_score": []}

    def _create_individual(self) -> dict:
        """Create a random individual (parameter set)."""
        return {
            key: values.rvs(random_state=self.random_state)
            if hasattr(values, "rvs")
            else random.choice(values)
            for key, values in self.param_space.items()
        }

    def _crossover(self, parent1: dict, parent2: dict) -> tuple[dict, dict]:
        """Create two offspring from two parents using uniform crossover."""
        child1, child2 = deepcopy(parent1), deepcopy(parent2)

        for key in parent1.keys():
            if random.random() < 0.5:
                child1[key], child2[key] = child2[key], child1[key]

        return child1, child2

    def _mutate(self, individual: dict) -> dict:
        """Mutate an individual by randomly changing some parameters."""
        mutated = deepcopy(individual)

        for key in individual.keys():
            if random.random() < self.mutation_rate:
                mutated[key] = (
                    self.param_space[key].rvs(random_state=self.random_state)
                    if hasattr(self.param_space[key], "rvs")
                    else random.choice(self.param_space[key])
                )
        return mutated

    def _tournament_selection(
        self, population: list, fitness_scores: list, tournament_size: int = 3
    ) -> dict:
        """Select an individual using tournament selection."""
        tournament_indices = random.sample(
            range(len(population)), min(tournament_size, len(population))
        )
        return population[
            tournament_indices[
                np.argmax([fitness_scores[i] for i in tournament_indices])
            ]
        ]

    def _evaluate_params(self, params: dict, X: DataFrame, y: Series) -> float:
        """Evaluate parameter configuration using cross-validation."""
        return np.mean(
            cross_val_score(
                OneClassSVM(**params, kernel="rbf"), X, y, cv=4, scoring=score_function
            )
        )

    def fit(self, X: DataFrame, y: Series):
        """Fit the genetic algorithm search."""
        random.seed(self.random_state)
        np.random.seed(self.random_state)
        population = [self._create_individual() for _ in range(self.population_size)]

        for generation in range(self.n_generations):
            print(f"Evaluating Generation {generation}")
            fitness_scores = []
            for individual in population:
                score = self._evaluate_params(individual, X, y)
                fitness_scores.append(score)
                self.cv_results_["mean_test_score"].append(score)

                if score > self.best_score_:
                    self.best_params_ = deepcopy(individual)
                    self.best_score_ = score

            new_population = [
                deepcopy(population[idx])
                for idx in np.argsort(fitness_scores)[-self.elite_size :]
            ]

            while len(new_population) < self.population_size:
                parent1 = self._tournament_selection(population, fitness_scores)
                parent2 = self._tournament_selection(population, fitness_scores)

                if random.random() < self.crossover_rate:
                    child1, child2 = self._crossover(parent1, parent2)
                else:
                    child1, child2 = deepcopy(parent1), deepcopy(parent2)

                new_population.extend([self._mutate(child1), self._mutate(child2)])

            population = new_population[: self.population_size]

        return self


class BayesianOptimizationSearch:
    """Wrapper for Bayesian Optimization to match the interface of other search methods."""

    def __init__(self, param_space: dict, random_state: int = 42):
        self.param_space = param_space
        self.random_state = random_state
        self.best_params_ = None
        self.best_score_ = -np.inf
        self.cv_results_ = {"mean_test_score": []}
        self.optimizer = None

    def fit(self, X: DataFrame, y: Series, n_iter: int = 100):
        """Fit the Bayesian optimization search."""
        global training_data, train_targets, testing_data, test_targets
        training_data, train_targets = X, y

        self.optimizer = BayesianOptimization(
            objective_function_bayesian,
            self.param_space,
            random_state=self.random_state,
        )

        if not os.path.exists(LOGS_PATH_BAYESIAN):
            with open(LOGS_PATH_BAYESIAN, "w") as fp:
                pass

        self.optimizer.subscribe(
            Events.OPTIMIZATION_STEP, JSONLogger(LOGS_PATH_BAYESIAN, False)
        )

        # Load existing logs if available
        if os.path.exists(LOGS_PATH_BAYESIAN):
            load_logs(self.optimizer, logs=[LOGS_PATH_BAYESIAN])

        # Perform optimization
        self.optimizer.maximize(init_points=25, n_iter=n_iter - 25)

        # Extract results
        self.best_params_ = self.optimizer.max["params"]
        self.best_score_ = self.optimizer.max["target"]

        # Extract all scores for cv_results_
        self.cv_results_["mean_test_score"] = [
            res["target"] for res in self.optimizer.res
        ]

        return self

In [5]:
def get_param_grid(
    search_method: Literal[
        "Grid", "Random", "SimulatedAnnealing", "GeneticAlgorithm", "Bayesian"
    ],
    use_log_dist: bool = False,
) -> ParamGrid:
    if search_method == "Grid":
        return {
            "nu": [0.01, 0.05, 0.1, 0.25],
            "gamma": ["scale", "auto", 0.001, 0.01, 0.1],
            "tol": [1e-1, 1e-2, 1e-3, 1e-4, 1e-5],
        }
    elif search_method in ["Random", "SimulatedAnnealing", "GeneticAlgorithm"]:
        if use_log_dist:
            return {
                "nu": loguniform(0.001, 0.3),
                "gamma": loguniform(1e-4, 10),
                "tol": loguniform(1e-6, 1e-1),
            }
        else:
            return {
                "nu": [0.01, 0.025, 0.05, 0.75, 0.1, 0.2, 0.3, 0.4, 0.5],
                "gamma": ["scale", "auto", 0.001, 0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1],
                "tol": [1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
            }
    elif search_method == "Bayesian":
        return {"nu": (0.01, 0.5), "gamma": (1e-4, 1), "tol": (1e-5, 1e-1)}


def update_train_vars(
    i: int, activities: ndarray
) -> tuple[DataFrame, Series, DataFrame, Series]:
    training = (
        X_train[X_train["activity"].isin(activities[:i])]
        .groupby("activity")
        .head(MIN_SAMPLES)
    )
    testing = X_test[X_test["activity"] == activities[i]].head(MIN_SAMPLES)
    training.loc[:, "isNovelty"], testing.loc[:, "isNovelty"] = False, True
    novelty = concat(
        [testing, training.sample(n=int(0.15 * len(training)), random_state=42)]
    )
    return (
        training.drop(columns=["isNovelty"]),
        training["isNovelty"],
        novelty.drop(columns=["isNovelty"]),
        novelty["isNovelty"],
    )


def train_search_method(
    training_data: DataFrame,
    train_targets: Series,
    search_type: Literal[
        "Grid", "Random", "SimulatedAnnealing", "GeneticAlgorithm", "Bayesian"
    ],
    params: dict[str, list],
    scoring: Callable,
    n_iter: int | None = 100,
    cv: int = 4,
    verbose: int = 1,
    random_state: int = 42,
) -> (
    RandomizedSearchCV
    | GridSearchCV
    | SimulatedAnnealingSearch
    | GeneticAlgorithmSearch
    | BayesianOptimizationSearch
):
    if search_type == "SimulatedAnnealing":
        return SimulatedAnnealingSearch(
            param_space=params,
            n_iter=n_iter or 100,
            initial_temp=1.0,
            cooling_rate=0.95,
            random_state=random_state,
        ).fit(training_data, train_targets)

    elif search_type == "GeneticAlgorithm":
        population_size = min(20, n_iter // 5) if n_iter else 20
        return GeneticAlgorithmSearch(
            param_space=params,
            population_size=population_size,
            n_generations=(n_iter // population_size) if n_iter else 5,
            mutation_rate=0.1,
            crossover_rate=0.8,
            random_state=random_state,
        ).fit(training_data, train_targets)

    elif search_type == "Bayesian":
        return BayesianOptimizationSearch(
            param_space=params,
            random_state=random_state,
        ).fit(training_data, train_targets, n_iter or 100)

    else:
        search_cls = RandomizedSearchCV if search_type == "Random" else GridSearchCV
        search_kwargs = {
            f"param_{'distributions' if search_type == 'Random' else 'grid'}": params,
            "estimator": OneClassSVM(kernel="rbf"),
            "scoring": scoring,
            "cv": cv,
            "verbose": verbose,
            "error_score": "raise",
        }
        if search_type == "Random" and n_iter:
            search_kwargs.update({"n_iter": n_iter, "random_state": random_state})
        return search_cls(**search_kwargs).fit(training_data, train_targets)

In [6]:
def compare_search_methods(
    grid_scores: list[float], random_scores: list[float]
) -> dict[str, float]:
    """Statistical comparison of search methods using Wilcoxon signed-rank test."""
    if len(grid_scores) != len(random_scores):
        raise ValueError("Score arrays must have equal length")

    statistic, p_value = wilcoxon(grid_scores, random_scores, alternative="two-sided")

    return {
        "wilcoxon_statistic": statistic,
        "p_value": p_value,
        "grid_mean": np.mean(grid_scores),
        "random_mean": np.mean(random_scores),
        "effect_size": (np.mean(grid_scores) - np.mean(random_scores))
        / np.std(grid_scores + random_scores),
    }


def update_params_grid(
    cv_results: dict[str, tuple], og_param_grid: ParamGrid
) -> ParamGrid:
    params = ["gamma", "nu", "tol"]
    top_entries = (
        DataFrame(
            zip(
                cv_results["rank_test_score"],
                cv_results["param_gamma"],
                cv_results["param_nu"],
                cv_results["param_tol"],
            ),
            columns=["rank_test_score", "gamma", "nu", "tol"],
        )
        .sort_values("rank_test_score")
        .head(NUM_TRIALS)
    )

    if len(top_entries) == NUM_TRIALS:
        print("Detected potential parameter space stagnation, diversifying...")

        current_params = {col: set(top_entries[col]) for col in params}
        unused_params = {
            param: list(set(og_param_grid[param]) - current_params[param])
            for param in params
        }

        for param in params:
            if unused_params[param]:
                current_unique = list(dict.fromkeys(top_entries[param]))
                if len(current_unique) > 1 and unused_params[param]:
                    current_unique = current_unique[:-1] + [unused_params[param][0]]
                elif unused_params[param]:
                    current_unique.append(unused_params[param][0])

                top_entries.loc[
                    top_entries[param] == list(dict.fromkeys(top_entries[param]))[-1],
                    param,
                ] = unused_params[param][0]

    exmp = {col: list(dict.fromkeys(top_entries[col])) for col in params}
    cartesian_size = len(exmp["gamma"]) * len(exmp["nu"]) * len(exmp["tol"])

    if cartesian_size > NUM_TRIALS:
        while cartesian_size > NUM_TRIALS:
            best_reduction = None
            best_param = None

            for param in params:
                if len(exmp[param]) > 1:
                    temp_sizes = [
                        len(exmp[p]) if p != param else len(exmp[p]) - 1 for p in params
                    ]
                    new_size = temp_sizes[0] * temp_sizes[1] * temp_sizes[2]
                    if new_size >= NUM_TRIALS and (
                        best_reduction is None or new_size < best_reduction
                    ):
                        best_reduction = new_size
                        best_param = param

            if best_param is None:
                param_lengths = [(param, len(exmp[param])) for param in params]
                param_lengths.sort(key=lambda x: x[1], reverse=True)
                best_param = param_lengths[0][0]

            if len(exmp[best_param]) > 1:
                exmp[best_param] = exmp[best_param][:-1]

            cartesian_size = len(exmp["gamma"]) * len(exmp["nu"]) * len(exmp["tol"])
            if all(len(exmp[param]) == 1 for param in params):
                break

    assert cartesian_size == NUM_TRIALS, "reducing the params space failed"
    print(f"dict of len {cartesian_size} :", exmp)
    return exmp


def eval_search_method(
    activities: ndarray,
    search_name: Literal[
        "Grid", "Random", "SimulatedAnnealing", "GeneticAlgorithm", "Bayesian"
    ],
    use_log_dist: bool = False,
) -> SearchResult:
    dist = get_param_grid(search_name, use_log_dist)
    MAXIMAZED = False
    BEST_SCORES = None

    global testing_data, test_targets, logger
    start_time = time()

    for i in range(1, len(activities)):
        training_data, train_targets, test_data, testing_targets = update_train_vars(
            i, activities
        )
        testing_data = test_data
        test_targets = testing_targets
        print(f"Training for activities {activities[:i]}")

        if not MAXIMAZED:
            search_method = train_search_method(
                training_data=training_data,
                train_targets=train_targets,
                search_type=search_name,
                params=dist,
                scoring=score_function,
            )
            BEST_SCORES = search_method
            MAXIMAZED = True
        else:
            print(f"Already maximized, suggesting new {NUM_TRIALS} points")
            if search_name in ["SimulatedAnnealing", "GeneticAlgorithm", "Bayesian"]:
                search_method = train_search_method(
                    training_data=training_data,
                    train_targets=train_targets,
                    search_type=search_name,
                    params=dist,
                    scoring=score_function,
                    n_iter=NUM_TRIALS,
                )
            else:
                search_method = train_search_method(
                    training_data=training_data,
                    train_targets=train_targets,
                    search_type=search_name,
                    params=update_params_grid(search_method.cv_results_, dist)
                    if search_name == "Grid"
                    else dist,
                    scoring=score_function,
                    n_iter=NUM_TRIALS if search_name == "Random" else None,
                )

            if search_method.best_score_ > BEST_SCORES.best_score_:
                BEST_SCORES = search_method

        print(f"{search_name} Search Best Params:", search_method.best_params_)

    return SearchResult(
        method=search_name + "_search",
        best_params=BEST_SCORES.best_params_,
        best_score=BEST_SCORES.best_score_,
        cv_scores=BEST_SCORES.cv_results_["mean_test_score"].tolist()
        if hasattr(BEST_SCORES.cv_results_["mean_test_score"], "tolist")
        else BEST_SCORES.cv_results_["mean_test_score"],
        fit_time=time() - start_time,
        n_evaluations=len(BEST_SCORES.cv_results_["mean_test_score"]),
    )

In [ ]:
activities = X_train["activity"].unique()

# Run all search methods
logger = configure_file_logger("../reports/logs_grid.log")
grid_result = eval_search_method(activities, "Grid")

logger = configure_file_logger("../reports/logs_rand.log")
rand_result = eval_search_method(activities, "Random")

logger = configure_file_logger("../reports/logs_rand_log.log")
rand_log_result = eval_search_method(activities, "Random", True)

logger = configure_file_logger("../reports/logs_sian.log")
sa_result = eval_search_method(activities, "SimulatedAnnealing", True)

logger = configure_file_logger("../reports/logs_geal.log")
ga_result = eval_search_method(activities, "GeneticAlgorithm", True)

logger = configure_file_logger("../reports/logs_bayesian.log")
bayesian_result = eval_search_method(activities, "Bayesian", True)

# Save comparison results
with open("../conf/test_results.json", "w") as file:
    file.write(
        dumps(
            [
                {
                    "test": "Grid x Rand",
                    **compare_search_methods(
                        grid_result.cv_scores, rand_result.cv_scores
                    ),
                },
                {
                    "test": "Grid x SA",
                    **compare_search_methods(
                        grid_result.cv_scores, sa_result.cv_scores
                    ),
                },
                {
                    "test": "Grid x GA",
                    **compare_search_methods(
                        grid_result.cv_scores, ga_result.cv_scores
                    ),
                },
                {
                    "test": "Grid x Bayesian",
                    **compare_search_methods(
                        grid_result.cv_scores, bayesian_result.cv_scores
                    ),
                },
                {
                    "test": "Rand x SA",
                    **compare_search_methods(
                        rand_result.cv_scores, sa_result.cv_scores
                    ),
                },
                {
                    "test": "Rand x GA",
                    **compare_search_methods(
                        rand_result.cv_scores, ga_result.cv_scores
                    ),
                },
                {
                    "test": "Rand x Bayesian",
                    **compare_search_methods(
                        rand_result.cv_scores, bayesian_result.cv_scores
                    ),
                },
                {
                    "test": "SA x GA",
                    **compare_search_methods(sa_result.cv_scores, ga_result.cv_scores),
                },
                {
                    "test": "SA x Bayesian",
                    **compare_search_methods(
                        sa_result.cv_scores, bayesian_result.cv_scores
                    ),
                },
                {
                    "test": "GA x Bayesian",
                    **compare_search_methods(
                        ga_result.cv_scores, bayesian_result.cv_scores
                    ),
                },
            ]
        )
    )

Training for activities [1]
Fitting 4 folds for each of 100 candidates, totalling 400 fits
Grid Search Best Params: {'gamma': 'scale', 'nu': 0.01, 'tol': 0.001}
Training for activities [1 2]
Already maximized, suggesting new 10 points
Detected potential parameter space stagnation, diversifying...
dict of len 10 : {'gamma': [0.1], 'nu': [0.01, 0.25], 'tol': [0.001, 0.0001, 0.01, 1e-05, 0.1]}
Fitting 4 folds for each of 10 candidates, totalling 40 fits
Grid Search Best Params: {'gamma': 0.1, 'nu': 0.01, 'tol': 1e-05}
Training for activities [1 2 3]
Already maximized, suggesting new 10 points
Detected potential parameter space stagnation, diversifying...


C:\Users\VDUART10\AppData\Local\Temp\ipykernel_33056\3122073337.py:55: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'auto' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  top_entries.loc[


dict of len 10 : {'gamma': ['auto'], 'nu': [0.01, 0.05], 'tol': [1e-05, 0.0001, 0.001, 0.01, 0.1]}
Fitting 4 folds for each of 10 candidates, totalling 40 fits
Grid Search Best Params: {'gamma': 'auto', 'nu': 0.01, 'tol': 1e-05}
Training for activities [1 2 3 4]
Already maximized, suggesting new 10 points
Detected potential parameter space stagnation, diversifying...
dict of len 10 : {'gamma': [0.1], 'nu': [0.01, 0.25], 'tol': [1e-05, 0.0001, 0.001, 0.01, 0.1]}
Fitting 4 folds for each of 10 candidates, totalling 40 fits
Grid Search Best Params: {'gamma': 0.1, 'nu': 0.01, 'tol': 0.0001}
Training for activities [1 2 3 4 5]
Already maximized, suggesting new 10 points
Detected potential parameter space stagnation, diversifying...


C:\Users\VDUART10\AppData\Local\Temp\ipykernel_33056\3122073337.py:55: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'auto' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  top_entries.loc[


dict of len 10 : {'gamma': ['auto'], 'nu': [0.01, 0.05], 'tol': [0.0001, 1e-05, 0.01, 0.001, 0.1]}
Fitting 4 folds for each of 10 candidates, totalling 40 fits
Grid Search Best Params: {'gamma': 'auto', 'nu': 0.05, 'tol': 1e-05}
Training for activities [1 2 3 4 5 6]
Already maximized, suggesting new 10 points
Detected potential parameter space stagnation, diversifying...
dict of len 10 : {'gamma': [0.1], 'nu': [0.05, 0.25], 'tol': [1e-05, 0.0001, 0.001, 0.01, 0.1]}
Fitting 4 folds for each of 10 candidates, totalling 40 fits
Grid Search Best Params: {'gamma': 0.1, 'nu': 0.05, 'tol': 0.001}
Training for activities [1 2 3 4 5 6 7]
Already maximized, suggesting new 10 points
Detected potential parameter space stagnation, diversifying...


...

C:\Users\VDUART10\AppData\Local\Temp\ipykernel_33056\3122073337.py:55: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'auto' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  top_entries.loc[


dict of len 10 : {'gamma': ['auto'], 'nu': [0.05, 0.1], 'tol': [1e-05, 0.0001, 0.001, 0.01, 0.1]}
Fitting 4 folds for each of 10 candidates, totalling 40 fits
Grid Search Best Params: {'gamma': 'auto', 'nu': 0.05, 'tol': 1e-05}
Training for activities [ 1  2  3  4  5  6  7 12 13 16]
Already maximized, suggesting new 10 points
Detected potential parameter space stagnation, diversifying...
dict of len 10 : {'gamma': [0.1], 'nu': [0.05, 0.25], 'tol': [1e-05, 0.0001, 0.001, 0.01, 0.1]}
Fitting 4 folds for each of 10 candidates, totalling 40 fits
Grid Search Best Params: {'gamma': 0.1, 'nu': 0.05, 'tol': 1e-05}
Training for activities [ 1  2  3  4  5  6  7 12 13 16 17]
Already maximized, suggesting new 10 points
Detected potential parameter space stagnation, diversifying...


C:\Users\VDUART10\AppData\Local\Temp\ipykernel_33056\3122073337.py:55: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'auto' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  top_entries.loc[


dict of len 10 : {'gamma': ['auto'], 'nu': [0.05, 0.1], 'tol': [1e-05, 0.0001, 0.001, 0.01, 0.1]}
Fitting 4 folds for each of 10 candidates, totalling 40 fits
Grid Search Best Params: {'gamma': 'auto', 'nu': 0.1, 'tol': 0.0001}
Training for activities [1]
Fitting 4 folds for each of 100 candidates, totalling 400 fits
Random Search Best Params: {'tol': 1e-05, 'nu': 0.01, 'gamma': 'scale'}
Training for activities [1 2]
Already maximized, suggesting new 10 points
Fitting 4 folds for each of 10 candidates, totalling 40 fits
Random Search Best Params: {'tol': 0.001, 'nu': 0.1, 'gamma': 0.01}
Training for activities [1 2 3]
Already maximized, suggesting new 10 points
Fitting 4 folds for each of 10 candidates, totalling 40 fits
Random Search Best Params: {'tol': 1e-05, 'nu': 0.3, 'gamma': 'scale'}
Training for activities [1 2 3 4]
Already maximized, suggesting new 10 points
Fitting 4 folds for each of 10 candidates, totalling 40 fits
Random Search Best Params: {'tol': 0.001, 'nu': 0.1, 'gamma

ValueError: Score arrays must have equal length